# 🚀 HFANet Training - CS415 Project

**Branch:** `hfa_net`

---

## Step 1: Setup - Clone & Branch

In [1]:
# --- CONFIGURATION ---
REPO_URL = "https://github.com/melihkn/CS415-PROJECT-CODES.git"
BRANCH = "hfa_net"  # <--- HFANet branch
PROJECT_PATH = "/content/drive/MyDrive/CS415/Project"
DATASET_PATH = "/content/drive/MyDrive/dataset"

# --- 1. MOUNT DRIVE ---
from google.colab import drive
import os
import sys

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- 2. NAVIGATE TO PROJECT FOLDER ---
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)
    print(f"Created folder: {PROJECT_PATH}")

%cd "{PROJECT_PATH}"

# --- 3. CLONE/PULL & SELECT BRANCH ---
repo_name = REPO_URL.split("/")[-1].replace(".git", "")

if not os.path.exists(repo_name):
    print(f"Cloning {BRANCH} branch...")
    !git clone -b "{BRANCH}" "{REPO_URL}"
else:
    print("Repo exists. Updating and switching branches...")
    %cd "{repo_name}"
    !git fetch origin
    !git checkout "{BRANCH}"
    !git pull origin "{BRANCH}"
    %cd ..

# --- 4. ADD TO PYTHON PATH ---
sys.path.append(os.path.join(PROJECT_PATH, repo_name))

print(f"\n✓ SUCCESS: Branch '{BRANCH}' ready at {PROJECT_PATH}/{repo_name}")

Mounted at /content/drive
/content/drive/MyDrive/CS415/Project
Cloning hfa_net branch...
Cloning into 'CS415-PROJECT-CODES'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 258 (delta 126), reused 202 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (258/258), 103.64 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (126/126), done.

✓ SUCCESS: Branch 'hfa_net' ready at /content/drive/MyDrive/CS415/Project/CS415-PROJECT-CODES


## Step 2: Change Directory to Project

In [2]:
os.chdir('/content/drive/MyDrive/CS415/Project/CS415-PROJECT-CODES')
print(f"Current directory: {os.getcwd()}")
!ls -la

Current directory: /content/drive/MyDrive/CS415/Project/CS415-PROJECT-CODES
total 47
drwx------ 2 root root  4096 Dec 27 17:25 data
drwx------ 8 root root  4096 Dec 27 17:25 .git
-rw------- 1 root root   462 Dec 27 17:25 .gitignore
drwx------ 4 root root  4096 Dec 27 17:25 models
drwx------ 2 root root  4096 Dec 27 17:25 notebooks
-rw------- 1 root root  7891 Dec 27 17:25 README.md
-rw------- 1 root root    84 Dec 27 17:25 requirements.txt
-rw------- 1 root root  6361 Dec 27 17:25 test.py
-rw------- 1 root root 11512 Dec 27 17:25 train.py
drwx------ 3 root root  4096 Dec 27 17:25 utils


## Step 3: Install Requirements

In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.6 MB/s eta 0:00:00


## Step 4: Link Dataset

In [4]:
DATASET_PATH = "/content/drive/MyDrive/dataset"

# Create symlink if it doesn't exist
if not os.path.exists('./dataset'):
    !ln -s "{DATASET_PATH}" ./dataset
    print(f"✓ Dataset linked: {DATASET_PATH} -> ./dataset")
else:
    print("Dataset link already exists")

# Check dataset contents
print("\nDataset contents:")
!ls ./dataset
print("\nTrain folder:")
!ls ./dataset/train

✓ Dataset linked: /content/drive/MyDrive/dataset -> ./dataset

Dataset contents:
DS_Store  test	train  val

Train folder:
A  B  label


## Step 5: Test Imports

In [5]:
# Import test
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Project imports
from data.dataset import get_dataloader
from models.HFANet.hfanet import HFANet, HFANet_timm
from utils.losses import DiceLoss, FocalLoss
from utils.training import train_one_epoch, CombinedLoss
from utils.evaluation import validate, evaluate_on_loader

print("\n✓ All imports successful!")

PyTorch: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB

✓ All imports successful!


## Step 6: Train HFANet (smp backbone)

In [12]:
!python train.py \
    --model hfanet \
    --backbone resnet50 \
    --data_dir ./dataset \
    --img_size 512 \
    --batch_size 8 \
    --epochs 150 \
    --lr 3e-4 \
    --weight_decay 1e-5 \
    --loss bce+focal \
    --optimizer adamw \
    --scheduler cosine \
    --patience 20


TRAINING CONFIGURATION
Device: cuda
Model: HFANet with backbone=resnet50

Loading data from: ./dataset
Train batches: 55, Val batches: 8
Loss function: bce+focal
Optimizer: adamw (lr=0.0003, weight_decay=1e-05)
Scheduler: cosine

STARTING TRAINING

Epoch 1 [Train]: 100% 55/55 [00:17<00:00,  3.20it/s, loss=0.2953, IoU=0.2457, F1=0.3944]
Validating: 100% 8/8 [00:02<00:00,  3.28it/s, val_loss=0.2012, val_IoU=0.1208, val_F1=0.2156]

Epoch [1/150] Summary:
  Train - Loss: 0.3890 | IoU: 0.1303 | F1: 0.2306
  Val   - Loss: 0.2090 | IoU: 0.1457 | F1: 0.2544
  ✓ New best model saved! (Val IoU: 0.1457)

Epoch 2 [Train]: 100% 55/55 [00:24<00:00,  2.24it/s, loss=0.1353, IoU=0.5882, F1=0.7407]
Validating: 100% 8/8 [00:02<00:00,  3.26it/s, val_loss=0.1307, val_IoU=0.5460, val_F1=0.7063]

Epoch [2/150] Summary:
  Train - Loss: 0.1802 | IoU: 0.4281 | F1: 0.5996
  Val   - Loss: 0.1387 | IoU: 0.5052 | F1: 0.6713
  ✓ New best model saved! (Val IoU: 0.5052)

Epoch 3 [Train]: 100% 55/55 [00:24<00:00,  2.2

## Step 7: Test Best Model

In [15]:
!python test.py \
    --model hfanet \
    --backbone resnet50 \
    --checkpoint ./checkpoints/best_hfanet.pth \
    --data_dir ./dataset \
    --img_size 512 \
    --output_dir ./results/hfanet \
    --save_predictions \
    --save_pr_curve


TEST CONFIGURATION
Device: cuda
Model: hfanet
Checkpoint: ./checkpoints/best_hfanet.pth
Loading checkpoint from: ./checkpoints/best_hfanet.pth
  Loaded from epoch 67
  Val IoU: 0.782638589002401

Loading test data from: ./dataset
Test batches: 128

STARTING EVALUATION

Evaluating [Test]: 100% 128/128 [00:05<00:00, 23.38it/s, IoU=0.7750]

TEST RESULTS
  IoU        : 0.7750
  F1-score   : 0.8733
  Precision  : 0.9102
  Recall     : 0.8391
  AP (PR AUC): 0.9471
--------------------------------------------------
  TP: 1434350  |  FP: 141439
  FN: 274942  |  TN: 31703701

PR curve saved to: ./results/hfanet/pr_curve.png
Metrics saved to: ./results/hfanet/metrics.json


## Step 8: Copy Results to Drive

In [16]:
# Checkpoint'ları Drive'a kopyala (yedek)
!cp -r ./checkpoints /content/drive/MyDrive/CS415/Project/checkpoints_backup
!cp -r ./results /content/drive/MyDrive/CS415/Project/results_backup

print("✓ Results backed up to Drive!")

✓ Results backed up to Drive!
